In [1]:
# Import Numpy And Pandas Library

import pandas as pd
import numpy as np

In [ ]:
# Import Data From CSV File
df = pd.read_csv("orders.csv")

In [ ]:
#We are interested in “MIS” products and Complete Status Stocks

df = df[(df["Status"]=="COMPLETE") & (df["Product"]=="MIS")]

In [ ]:
# Data Cleaning
df.drop(columns="Time",inplace=True)
df["Qty."]=df["Qty."].apply(lambda x: x.split("/")[0]).astype(int)

## 1) Different types of charges for Individual trade

In [2]:
df["Turnover"] = df["Qty."]*df["Avg. price"]
df["Brokerage"] = round(df["Turnover"]*0.03/100,2).apply(lambda x :20 if x>20 else x)
df["STT/CTT"] = round(df["Turnover"]*0.025/100,2)
df.loc[df['Type'] == 'BUY', 'STT/CTT'] = 0
df["ETC"] = round(df["Turnover"]*0.00325/100,2)
df["SEBI"] = round(10/10000000*df["Turnover"],2)
df["GST"] = round(((df["Brokerage"]+df["SEBI"]+df["ETC"])*18)/100,2)
df["Stamp Charges"] = round(df["Turnover"]*0.003/100,2)
df.loc[df['Type'] == 'SELL',"Stamp Charges" ] = 0
df["Total Charges"] = round(df["Brokerage"]+df["STT/CTT"]+df["ETC"]+df["SEBI"]+df["GST"]+df["Stamp Charges"],2)
df

,Type,Instrument,Product,Qty.,Avg. price,Status,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp Charges,Total Charges
1,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE,125700.0,20,0.00,4.09,0.13,4.36,3.77,32.35
3,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE,122637.5,20,0.00,3.99,0.12,4.34,3.68,32.13
5,SELL,TATAMOTORS,MIS,250,492.10,COMPLETE,123025.0,20,30.76,4.00,0.12,4.34,0.00,59.22
7,SELL,ASHOKLEY,MIS,1000,125.96,COMPLETE,125960.0,20,31.49,4.09,0.13,4.36,0.00,60.07
8,BUY,ASHOKLEY,MIS,2000,125.70,COMPLETE,251400.0,20,0.00,8.17,0.25,5.12,7.54,41.08
9,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20,62.98,8.19,0.25,5.12,0.00,96.54
11,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20,62.98,8.19,0.25,5.12,0.00,96.54
12,BUY,ASHOKLEY,MIS,1000,125.60,COMPLETE,125600.0,20,0.00,4.08,0.13,4.36,3.77,32.34
13,BUY,ASHOKLEY,MIS,1000,125.65,COMPLETE,125650.0,20,0.00,4.08,0.13,4.36,3.77,32.34


## 2) Stock wise and Type wise analysis with weighted Avg. price & calculated charges

In [3]:
stock_type_wise = df.groupby(['Instrument', 'Type']).agg({
        'Qty.': 'sum',
        'Avg. price': 'mean',
        'Turnover': 'sum',
        'Brokerage': 'sum',
        'STT/CTT': 'sum',
        'ETC': 'sum',
        'SEBI': 'sum',
        'GST': 'sum',
        'Stamp Charges': 'sum',
        'Total Charges': 'sum'}).reset_index()
stock_type_wise

,Instrument,Type,Qty.,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp Charges,Total Charges
0,ASHOKLEY,BUY,5000,125.662500,628350.0,80,0.00,20.42,0.64,18.20,18.85,138.11
1,ASHOKLEY,SELL,5000,125.953333,629760.0,60,157.45,20.47,0.63,14.60,0.00,253.15
2,TATAMOTORS,BUY,250,490.550000,122637.5,20,0.00,3.99,0.12,4.34,3.68,32.13
3,TATAMOTORS,SELL,250,492.100000,123025.0,20,30.76,4.00,0.12,4.34,0.00,59.22


## 3) Overall Summary of each Stocks

In [4]:
df1 = df.groupby('Instrument')['Total Charges'].sum().reset_index()
shares = df1['Instrument'].unique()
d = {}
for i in shares:
    buy = df[(df['Instrument'] == i) & (df['Type'] == 'BUY')]['Turnover'].sum()
    sell = df[(df['Instrument'] == i) & (df['Type'] == 'SELL')]['Turnover'].sum()
    d[i] = sell-buy
df1["Gross PnL"] = d.values()
df1["Net PnL"] = df1["Gross PnL"] - df1["Total Charges"]
df1['% Charges on Gross PnL'] = round((df1['Total Charges'] / df1['Gross PnL']) * 100,2)
df1

,Instrument,Total Charges,Gross PnL,Net PnL,% Charges on Gross PnL
0,ASHOKLEY,391.26,1410.0,1018.74,27.75
1,TATAMOTORS,91.35,387.5,296.15,23.57


In [5]:
with pd.ExcelWriter("Kite problem assisment.xlsx") as writer:
    df.to_excel(writer,sheet_name="Individual trade charge",index=False)
    stock_type_wise.to_excel(writer,sheet_name="stock_type_wise_charge",index=False)
    df1.to_excel(writer,sheet_name="overall_summary",index=False)